## Data analysis and visualisation with an Open AI assistant


#### Import libraries
Note: openai==1.2.4 was required here

In [1]:
import os
from dotenv import load_dotenv
from openai import OpenAI
from openai.types import Image, ImagesResponse
import json
from IPython.display import display, Markdown

#### Define a series of helper functions

In [2]:
# Capture json output from Assitants API
def show_json(obj):
    display(json.loads(obj.model_dump_json()))

# Print messages from threads so you can read them
def pretty_print(messages):
    print("# Messages")
    for m in messages:
        try:
            print(f"{m.role}: {m.content[0].text.value}")
        except:
            pass
    print()

# Output any image created by the assistant
def save_images(open_ai_file_name, desired_file_name):
    client = OpenAI()

    image_data = client.files.content(open_ai_file_name)
    image_data_bytes = image_data.read()

    with open(f"./{desired_file_name}.png", "wb") as file:
        file.write(image_data_bytes)

# Retrieve the thread from the assistants api
def get_response(thread):
    return client.beta.threads.messages.list(thread_id=thread.id, order="asc")

#### Define variables
Note: Open api key is called from .env file - you will need to create your own .env file and obtain your own key

In [3]:
load_dotenv()
OPENAI_MODEL = "gpt-4"
OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")
client = OpenAI()

#### Assign the input file
This is a file a downloaded from the World Bank Development Indicators dataset [WDI](https://databank.worldbank.org/source/world-development-indicators)

In [4]:

file_path = "./data/API_NY.GDP.PCAP.PP.CD_DS2_en_csv_v2_5728866.csv"

file = client.files.create(
  file=open(file_path, "rb"),
  purpose='assistants'
)

#### Create the assistant
This will also appear in Open AI UI for Assiants when you are logged in

In [5]:
assistant = client.beta.assistants.create(
  name="Data visualizer",
  description="You are great at creating beautiful data visualizations. You analyze data present in .csv files, understand trends, and come up with data visualizations relevant to those trends. You also share a brief text summary of the trends observed.",
  model="gpt-4-1106-preview",
  tools=[{"type": "code_interpreter"}],
  file_ids=[file.id]
)

#### Create a thread

In [6]:

thread = client.beta.threads.create()

#### Add a message to the thread
Note: Here I instruct the assitant to skip the forst 4 rows, previously it did work out that this needed to be done, but you use a lot of tokens and it doesn't always work

In [7]:
message = client.beta.threads.messages.create(
      thread_id=thread.id,
      role = "user",
      content = "Analyse the data for Switzerland and the UK. skip the first 4 rows when reading data in",
      file_ids= [file.id]
)

#### submit messages to the agent

In [8]:

run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id
)

#### retrieve messages from assistant

In [9]:
messages = client.beta.threads.messages.list(thread_id=thread.id)
show_json(messages)

{'data': [{'id': 'msg_xzhywYRBZ8X3Xl3MJM9A0IMP',
   'assistant_id': 'asst_9ClY18CzRXIW9Dy807YeaNPF',
   'content': [{'text': {'annotations': [],
      'value': 'The filtered data includes information for Switzerland and the United Kingdom, specifically showing the "GDP per capita, PPP (current international $)" indicator. We can see GDP per capita values from 1960 through to 2022, although the actual data reporting starts from a later year as the earlier columns contain NaN (missing values).\n\nWould you like a specific analysis or summary of this data, such as trends over time, a comparison between the two countries, or perhaps statistical summaries for a particular range of years? Please let me know how you\'d like to proceed.'},
     'type': 'text'}],
   'created_at': 1700139199,
   'file_ids': [],
   'metadata': {},
   'object': 'thread.message',
   'role': 'assistant',
   'run_id': 'run_w31jwAn3dGyIa8tMPpWBEd2o',
   'thread_id': 'thread_MjBrsnd8tW0mWErxGWQLA8Ca'},
  {'id': 'msg_n9

In [10]:
pretty_print(get_response(thread))

# Messages
user: Analyse the data for Switzerland and the UK. skip the first 4 rows when reading data in
assistant: It seems that there was an issue processing your file using both the Excel and CSV parsers. To further investigate the issue and determine the correct way to parse the content, I will now attempt to read the file using a generic text reader to examine the raw contents. This will help to identify the file format and structure. Let's proceed with that.
assistant: The file appears to be a CSV format and contains a header row followed by rows of comma-separated values. Interestingly, we can already see entries related to countries and various economic indicators, such as "GDP per capita, PPP (current international $)". For such a structured CSV file, we could use pandas `read_csv` with the proper options. Let me try to load the data once again, this time making sure to handle any Unicode decoding issues (as suggested by the presence of '\ufeff' at the start of the file) and s

#### Add further instructions

In [11]:
message = client.beta.threads.messages.create(
      thread_id=thread.id,
      role = "user",
      content = "Calculate annual average growth in GDP per capita for the UK and Switzerland for each decade from 1900 to 2020",
      file_ids= [file.id]
)

In [12]:
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id
)

In [13]:
messages = client.beta.threads.messages.list(thread_id=thread.id)
show_json(messages)

{'data': [{'id': 'msg_tGnk9JhcvxB0TxQdzX7NQl79',
   'assistant_id': 'asst_9ClY18CzRXIW9Dy807YeaNPF',
   'content': [{'text': {'annotations': [],
      'value': 'There was an error in the calculation due to a mismatch between the expected column labels and those present in the data. It seems I attempted to access a year, "2029", which is beyond the range of the data provided. Since the data only goes up to 2022, I will adjust the calculation to consider this and will compute the average annual growth rates for the complete decades that we have data for, namely the 1960s through the 2010s.\n\nLet me correct this and recalculate the growth rates.'},
     'type': 'text'}],
   'created_at': 1700139270,
   'file_ids': [],
   'metadata': {},
   'object': 'thread.message',
   'role': 'assistant',
   'run_id': 'run_TzsrrPyhzS6SaKqFmGjpMRlG',
   'thread_id': 'thread_MjBrsnd8tW0mWErxGWQLA8Ca'},
  {'id': 'msg_Zke35z9Xdc10OZcaoa14lxPg',
   'assistant_id': 'asst_9ClY18CzRXIW9Dy807YeaNPF',
   'conten

In [14]:
pretty_print(get_response(thread))

# Messages
user: Analyse the data for Switzerland and the UK. skip the first 4 rows when reading data in
assistant: It seems that there was an issue processing your file using both the Excel and CSV parsers. To further investigate the issue and determine the correct way to parse the content, I will now attempt to read the file using a generic text reader to examine the raw contents. This will help to identify the file format and structure. Let's proceed with that.
assistant: The file appears to be a CSV format and contains a header row followed by rows of comma-separated values. Interestingly, we can already see entries related to countries and various economic indicators, such as "GDP per capita, PPP (current international $)". For such a structured CSV file, we could use pandas `read_csv` with the proper options. Let me try to load the data once again, this time making sure to handle any Unicode decoding issues (as suggested by the presence of '\ufeff' at the start of the file) and s

In [15]:
message = client.beta.threads.messages.create(
      thread_id=thread.id,
      role = "user",
      content = "Plot a chart of GDP per Capita over time for Switzerland and the UK",
      file_ids= [file.id]
)
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id
)
messages = client.beta.threads.messages.list(thread_id=thread.id)
show_json(messages)

{'data': [{'id': 'msg_hrBvI0tFoWgUDq4k5dnlfZi0',
   'assistant_id': None,
   'content': [{'text': {'annotations': [],
      'value': 'Plot a chart of GDP per Capita over time for Switzerland and the UK'},
     'type': 'text'}],
   'created_at': 1700139348,
   'file_ids': ['file-zNbnuaQAtlv8GThQeOxTJnn3'],
   'metadata': {},
   'object': 'thread.message',
   'role': 'user',
   'run_id': None,
   'thread_id': 'thread_MjBrsnd8tW0mWErxGWQLA8Ca'},
  {'id': 'msg_e4N5IZPiJMxf1k55Z6UonWWe',
   'assistant_id': 'asst_9ClY18CzRXIW9Dy807YeaNPF',
   'content': [{'text': {'annotations': [],
      'value': 'Here are the calculated average annual growth rates of GDP per capita in percentage for Switzerland and the United Kingdom for the decades where we have complete data:\n\n- **1990s**:\n  - Switzerland: 2.01%\n  - United Kingdom: 3.67%\n\n- **2000s**:\n  - Switzerland: 3.74%\n  - United Kingdom: 2.83%\n\n- **2010s**:\n  - Switzerland: 2.93%\n  - United Kingdom: 3.03%\n\nPlease note that the data fr

#### Extracting images
1. Find the messages containing an image
2. extract the filename
3. Decode and save that image

In [16]:
messages = client.beta.threads.messages.list(thread_id=thread.id, order="asc")
for m in messages:
    print(m.content)

[MessageContentText(text=Text(annotations=[], value='Analyse the data for Switzerland and the UK. skip the first 4 rows when reading data in'), type='text')]
[MessageContentText(text=Text(annotations=[], value="It seems that there was an issue processing your file using both the Excel and CSV parsers. To further investigate the issue and determine the correct way to parse the content, I will now attempt to read the file using a generic text reader to examine the raw contents. This will help to identify the file format and structure. Let's proceed with that."), type='text')]
[MessageContentText(text=Text(annotations=[], value='The file appears to be a CSV format and contains a header row followed by rows of comma-separated values. Interestingly, we can already see entries related to countries and various economic indicators, such as "GDP per capita, PPP (current international $)". For such a structured CSV file, we could use pandas `read_csv` with the proper options. Let me try to load 

In [17]:
file_name = m.content[-2].image_file.file_id
print(m.content[-2].image_file.file_id)

file-Bz3Wb1Gf2LSHvtsDJiUztUHQ


In [18]:
save_images(file_name,"ch_uk_gdp_per_capita")

#### Add further instructions - Generate forecasts

In [19]:
message = client.beta.threads.messages.create(
      thread_id=thread.id,
      role = "user",
      content = "“Forecast GDP per Capita growth rates for the next five years",
      file_ids= [file.id]
)
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id
)
messages = client.beta.threads.messages.list(thread_id=thread.id)

In [25]:
messages = client.beta.threads.messages.list(thread_id=thread.id, order="asc")
for m in messages:
    print(m.content)

[MessageContentText(text=Text(annotations=[], value='Analyse the data for Switzerland and the UK. skip the first 4 rows when reading data in'), type='text')]
[MessageContentText(text=Text(annotations=[], value="It seems that there was an issue processing your file using both the Excel and CSV parsers. To further investigate the issue and determine the correct way to parse the content, I will now attempt to read the file using a generic text reader to examine the raw contents. This will help to identify the file format and structure. Let's proceed with that."), type='text')]
[MessageContentText(text=Text(annotations=[], value='The file appears to be a CSV format and contains a header row followed by rows of comma-separated values. Interestingly, we can already see entries related to countries and various economic indicators, such as "GDP per capita, PPP (current international $)". For such a structured CSV file, we could use pandas `read_csv` with the proper options. Let me try to load 

In [26]:
pretty_print(get_response(thread))

# Messages
user: Analyse the data for Switzerland and the UK. skip the first 4 rows when reading data in
assistant: It seems that there was an issue processing your file using both the Excel and CSV parsers. To further investigate the issue and determine the correct way to parse the content, I will now attempt to read the file using a generic text reader to examine the raw contents. This will help to identify the file format and structure. Let's proceed with that.
assistant: The file appears to be a CSV format and contains a header row followed by rows of comma-separated values. Interestingly, we can already see entries related to countries and various economic indicators, such as "GDP per capita, PPP (current international $)". For such a structured CSV file, we could use pandas `read_csv` with the proper options. Let me try to load the data once again, this time making sure to handle any Unicode decoding issues (as suggested by the presence of '\ufeff' at the start of the file) and s

In [27]:
message = client.beta.threads.messages.create(
      thread_id=thread.id,
      role = "user",
      content = "Plot charts showing the linear trend used to create the forecasts",
      file_ids= [file.id]
)
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id
)
messages = client.beta.threads.messages.list(thread_id=thread.id)

In [28]:
messages = client.beta.threads.messages.list(thread_id=thread.id, order="asc")
for m in messages:
    print(m.content)

[MessageContentText(text=Text(annotations=[], value='Analyse the data for Switzerland and the UK. skip the first 4 rows when reading data in'), type='text')]
[MessageContentText(text=Text(annotations=[], value="It seems that there was an issue processing your file using both the Excel and CSV parsers. To further investigate the issue and determine the correct way to parse the content, I will now attempt to read the file using a generic text reader to examine the raw contents. This will help to identify the file format and structure. Let's proceed with that."), type='text')]
[MessageContentText(text=Text(annotations=[], value='The file appears to be a CSV format and contains a header row followed by rows of comma-separated values. Interestingly, we can already see entries related to countries and various economic indicators, such as "GDP per capita, PPP (current international $)". For such a structured CSV file, we could use pandas `read_csv` with the proper options. Let me try to load 

In [29]:
m.content[-3].image_file.file_id

'file-h518gFn4Whm3jm1YyknMEYKM'

In [30]:
file_name_01 = m.content[-2].image_file.file_id
file_name_02 = m.content[-3].image_file.file_id
print(file_name_01)
print(file_name_02)

file-dMjZ9yZXKuLfPz39mlPehtiu
file-h518gFn4Whm3jm1YyknMEYKM


In [31]:
save_images(file_name_01,"uk_forecast")
save_images(file_name_02,"ch_forecast")